# Training a Simple LUTNN

In [1]:
import torch
import torch.optim as optim
from lutnn.lutlayer import LUTLayer, Aggregation

ModuleNotFoundError: No module named 'torch'

In [ ]:
from utils.mnist import load_mnist_dataset
train_loader, test_loader, input_dim_dataset, num_classes = load_mnist_dataset(batch_size=128, mnist20=True)

In [ ]:
# Define a simple LUTNN
class SimpleLUTNN(torch.nn.Module):
    def __init__(self):
        super(SimpleLUTNN, self).__init__()
        self.layer1 = LUTLayer(input_dim=input_dim_dataset, lut_size=6, n_luts=2048)
        self.layer2 = LUTLayer(input_dim=2048, lut_size=6, n_luts=4000)
        self.layer3 = Aggregation(num_classes=num_classes, tau = 10.)

    def forward(self, x):
        x = x.view(-1, 20*20)  # Flatten the input
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = SimpleLUTNN().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

cuda


In [ ]:
# Training loop
num_epochs = 4
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, target)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Epoch [1/4], Step [0/468], Loss: 2.3123
Epoch [1/4], Step [100/468], Loss: 0.4844
Epoch [1/4], Step [200/468], Loss: 0.2999
Epoch [1/4], Step [300/468], Loss: 0.2200
Epoch [1/4], Step [400/468], Loss: 0.2616
Epoch [2/4], Step [0/468], Loss: 0.1745
Epoch [2/4], Step [100/468], Loss: 0.1362
Epoch [2/4], Step [200/468], Loss: 0.1092
Epoch [2/4], Step [300/468], Loss: 0.1060
Epoch [2/4], Step [400/468], Loss: 0.2125
Epoch [3/4], Step [0/468], Loss: 0.1046
Epoch [3/4], Step [100/468], Loss: 0.0623
Epoch [3/4], Step [200/468], Loss: 0.0400
Epoch [3/4], Step [300/468], Loss: 0.0542
Epoch [3/4], Step [400/468], Loss: 0.0975
Epoch [4/4], Step [0/468], Loss: 0.0568
Epoch [4/4], Step [100/468], Loss: 0.0794
Epoch [4/4], Step [200/468], Loss: 0.0792
Epoch [4/4], Step [300/468], Loss: 0.1054
Epoch [4/4], Step [400/468], Loss: 0.0453
Test Accuracy: 97.30%
